# Tasks that 'extract_kpts' class follows


## extract_kpts.dataset()
    1) Iterate through the yes and no folder.
    2) Extract keypoints of of each image and store them in a numpy array
       of each class folder respectively.
## extract_kpts.inference_img()
    For inferencing on images.
## extract_kpts.inference_vid()
    For inferencing on videos.
**Note :** Below functions take large amount of time in creating the dataset (~ 25mins) and inferencing videos (~ 5mins). 


In [4]:
from imutils import paths
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
import cv2
import numpy as np
import pandas as pd
import os


class extract_kpts:


    def preprocess(img):
        frame = cv2.imread(img)
        frameCopy = np.copy(frame)
        frameWidth = frame.shape[1]
        frameHeight = frame.shape[0]
        aspect_ratio = frameWidth/frameHeight

        inHeight = 368
        inWidth = int(((aspect_ratio*inHeight)*8)//8)

        return inHeight, inWidth, frameWidth, frameHeight, frame
    def preprocess_vid(img):
        frameWidth = img.shape[1]
        frameHeight = img.shape[0]
        aspect_ratio = frameWidth/frameHeight

        inHeight = 368
        inWidth = int(((aspect_ratio*inHeight)*8)//8)

        return inHeight, inWidth, frameWidth, frameHeight


    def dataset(protoFile,weightsFile,path,threshold=0):
        points = []
        labels = []
        nPoints = 22
        imagePaths = list(paths.list_images(path))
        net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)
        # loop over the image paths
        for imagePath in imagePaths:
            # extract the class label from the filename
            label = imagePath.split(os.path.sep)[-2]
            # load the input image (224x224) and preprocess it
            img_h, img_w, frameWidth, frameHeight, img = extract_kpts.preprocess(imagePath)
            inpBlob = cv2.dnn.blobFromImage(img, 1.0 / 255, (img_w, img_h), (0, 0, 0), swapRB=False, crop=False)

            net.setInput(inpBlob)

            output = net.forward()  

            for i in range(nPoints):
                # confidence map of corresponding body's part.
                probMap = output[0, i, :, :]
                probMap = cv2.resize(probMap, (frameWidth, frameHeight))

                # Find global maxima of the probMap. to determine location of Keypoint
                minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)

                if prob > threshold :
                    # Add the point to the list if the probability is greater than the threshold
                    points.append((int(point[0]), int(point[1])))
                    if label == 'yes':
                        labels.append(1)
                    else:
                        labels.append(0)
                else :
                    points.append(None) 

        points = np.array(points)
        labels = np.array(labels)

        return points, labels
    
    def save_file():
        # 35,376
        # [0 : 17687] - 0
        # [17688 : ]  - 1
        # 35376/44 = 804
        # 804 * 2 = 1608

        protoFile = "hand/pose_deploy.prototxt"
        WeightsFile = "hand/pose_iter_102000.caffemodel"


        if os.path.isfile('final_dataset_sample.csv') is False: 

            points, labels = extract_kpts.dataset(protoFile,WeightsFile,path)
            data = pd.DataFrame()
            classes = pd.DataFrame(labels, columns=['class'])

            # Change the column names accordingly in future dataset
            points = pd.DataFrame(points, columns=['X','Y'])

            data = pd.concat([points, classes], axis=1)
            df = pd.DataFrame(data)
            
            # Making 44 columns(22*X,22*Y) using every 22 rows(samples)
            a = df[['X','Y']].to_numpy().reshape(-1, 44)

            df1 = pd.DataFrame(a)
            df1['class'] = df['class'].to_numpy().reshape(a.shape[0], -1)[:, 0]

            df1.to_csv('final_dataset_sample.csv',index=False)
    
    def inference_img(protoFile,weightsFile,imagePath):
        nPoints = 22
        threshold=0
        points=[]
        net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)
        img_h, img_w, frameWidth, frameHeight, img = extract_kpts.preprocess(imagePath)
        inpBlob = cv2.dnn.blobFromImage(img, 1.0 / 255, (img_w, img_h), (0, 0, 0), swapRB=False, crop=False)

        net.setInput(inpBlob)

        output = net.forward()  
        for i in range(nPoints):
            # confidence map of corresponding body's part.
            probMap = output[0, i, :, :]
            probMap = cv2.resize(probMap, (frameWidth, frameHeight))

            # Find global maxima of the probMap. to determine location of Keypoint
            minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)

            if prob > threshold :
                # Add the point to the list if the probability is greater than the threshold
                points.append((int(point[0]), int(point[1])))
        points = np.array(points)


        return points    
    
    def inference_vid(args,path,protoFile,weightsFile,vidPath,model):
        
        cap = cv2.VideoCapture(path)
        hasFrame, frame = cap.read()
        net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)
        nPoints = 22
        threshold=0
        points=[]      
        dir = os.path.join(args['output_dir']+'Output_vid.avi')
        vid_writer = cv2.VideoWriter(dir,cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame.shape[1],frame.shape[0]))
        while hasFrame:
                       
            hasFrame, frame = cap.read()
            img_h, img_w, frameWidth, frameHeight = extract_kpts.preprocess_vid(frame) 
            if not hasFrame:
                cv2.waitKey()
                break
            
            inpBlob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (img_w, img_h), (0, 0, 0), swapRB=False, crop=False)

            net.setInput(inpBlob)

            output = net.forward()  
            for i in range(nPoints):
                # confidence map of corresponding body's part.
                probMap = output[0, i, :, :]
                probMap = cv2.resize(probMap, (frameWidth, frameHeight))

                # Find global maxima of the probMap. to determine location of Keypoint
                minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)

                if prob > threshold :
                    # Add the point to the list if the probability is greater than the threshold
                    points.append((int(point[0]), int(point[1])))
                if ((i+1)%22==0):
                    points = np.array(points)
                    
                    points = pd.DataFrame(points, columns=['X','Y'])
                    df = pd.DataFrame(points)
                    a = df[['X','Y']].to_numpy().reshape(-1, 44)

                    df1 = pd.DataFrame(a)
                    points = []

                    frame=cv2.putText(frame,"Output : {}".format(np.argmax(model.predict(df1),axis=1)[0]),(0,30),
                    cv2.FONT_ITALIC,1.3,(255,0,0),3)
                    
            cv2.imshow('frame', frame)    
            
            key = cv2.waitKey(1) & 0xFF
            if key == ord("q"):
                break
            vid_writer.write(frame)
        cap.release()        
        cv2.destroyAllWindows()


# Reference codes

In [3]:
# from imutils import paths
# from tensorflow.keras.preprocessing.image import load_img
# from tensorflow.keras.preprocessing.image import img_to_array
# import cv2
# import numpy as np
# import pandas as pd
# import os


# def preprocess(img):
#     frame = cv2.imread(img)
#     frameCopy = np.copy(frame)
#     frameWidth = frame.shape[1]
#     frameHeight = frame.shape[0]
#     aspect_ratio = frameWidth/frameHeight

#     inHeight = 368
#     inWidth = int(((aspect_ratio*inHeight)*8)//8)
    
#     return inHeight, inWidth, frameWidth, frameHeight, frame


# def dataset(path):
#     protoFile = "hand/pose_deploy.prototxt"
#     weightsFile = "hand/pose_iter_102000.caffemodel"
#     imagePaths = list(paths.list_images(path))
#     threshold = 0
#     points = []
#     labels = []
#     nPoints = 22
#     net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)
#     # loop over the image paths
#     for imagePath in imagePaths:
#         # extract the class label from the filename
#         label = imagePath.split(os.path.sep)[-2]
#         # load the input image (224x224) and preprocess it
#         img_h, img_w, frameWidth, frameHeight, img = preprocess(imagePath)
#         inpBlob = cv2.dnn.blobFromImage(img, 1.0 / 255, (img_w, img_h), (0, 0, 0), swapRB=False, crop=False)

#         net.setInput(inpBlob)

#         output = net.forward()  

#         for i in range(nPoints):
#             # confidence map of corresponding body's part.
#             probMap = output[0, i, :, :]
#             probMap = cv2.resize(probMap, (frameWidth, frameHeight))

#             # Find global maxima of the probMap. to determine location of Keypoint
#             minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)

#             if prob > threshold :
#                 # Add the point to the list if the probability is greater than the threshold
#                 points.append((int(point[0]), int(point[1])))
#                 #cv2.putText(frameCopy, "{}".format(i), (int(point[0]), int(point[1])), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, lineType=cv2.LINE_AA)
                
#                 if label == 'yes':
#                     labels.append(1)
#                 else:
#                     labels.append(0)
#             else :
#                 points.append(None) 
        

#     points = np.array(points)
#     labels = np.array(labels)
    
#     return points, labels


In [ ]:
# def save_file():
#         protoFile = "hand/pose_deploy.prototxt"
#         WeightsFile = "hand/pose_iter_102000.caffemodel"
#         #     imagePaths = list(paths.list_images(path))
#         threshold = 0
#         if os.path.isfile('KPts_dataset_2.csv') is False:
#             path = os.getcwd()+'\\dataset'

#             dataset = extract_kpts.dataset(protoFile,WeightsFile,path)

#             # Extracting points and labels np arrays
#             points = dataset[0]
#             labels = dataset[1]

#             classes = labels.copy()
#             classes = pd.DataFrame(classes, columns=['class'])

#             # Change the column names accordingly in future dataset
#             data = pd.DataFrame(points, columns=['X','Y'])
#             # data.to_csv('data_XY.csv', index=False)

#             # Concatenating both classes and Keypoints and saving the dataset
#             dataset = pd.concat([data,classes],axis=1)
#             dataset.to_csv('KPts_dataset_2.csv',index=False)
        

In [5]:
# data = pd.read_csv('KPts_dataset_2.csv')



# df = pd.DataFrame(data)
# a = df[['X','Y']].to_numpy().reshape(-1, 44)
       
# df1 = pd.DataFrame(a)
# df1['class'] = df['class'].to_numpy().reshape(a.shape[0], -1)[:, 0]

# data_2 = pd.DataFrame(df1)
# dataset = pd.concat([data_1, data_2], axis=0)
# dataset.to_csv('final_dataset_sample.csv',index=False)